In [1]:
%run data_processing.py
# memory leaking!

24-01-08 22:11:06|MOD:data_processing|: Data Processing start!
24-01-08 22:11:06|MOD:data_processing|: 3 datas :['y', 'trade_day', 'trade_15m']


Mon Jan  8 22:11:06 2024 : y start ...
y blocks reading 10days/lag1 Data1D's ...... cost 12.34 secs
y blocks reading 20days/lag1 Data1D's ...... cost 9.55 secs
y blocks merging ...... cost 4.87 secs
y blocks process ...... cost 1.11 secs
y blocks saving  ...... cost 18.37 secs
y blocks norming ...... cost 0.00 secs
Mon Jan  8 22:11:52 2024 : y finished! Cost 46.25 Seconds
Mon Jan  8 22:11:52 2024 : trade_day start ...
trade_day blocks reading day/trade Data1D's ...... cost 24.59 secs
trade_day blocks merging ...... cost 0.13 secs
trade_day blocks process ...... cost 5.87 secs
trade_day blocks saving  ...... cost 23.85 secs
trade_day blocks norming ...... cost 24.47 secs
Mon Jan  8 22:13:11 2024 : trade_day finished! Cost 78.89 Seconds
Mon Jan  8 22:13:11 2024 : trade_15m start ...
trade_15m blocks reading 15min/trade Data1D's ...... cost 1386.24 secs
trade_15m blocks merging ...... cost 0.37 secs
trade_15m blocks process ...... cost 21.47 secs
trade_15m blocks saving  ...... cost 179.2

24-01-08 22:39:52|MOD:data_processing|: Data Processing Finished! Cost 1726.34 Seconds


... cost 13.04 secs
Mon Jan  8 22:39:52 2024 : trade_15m finished! Cost 1600.35 Seconds


In [3]:
import torch
for pos in torch.utils.data.BatchSampler(range(30),5,drop_last=False):
    print(pos)

[0, 1, 2, 3, 4]
[5, 6, 7, 8, 9]
[10, 11, 12, 13, 14]
[15, 16, 17, 18, 19]
[20, 21, 22, 23, 24]
[25, 26, 27, 28, 29]


In [8]:
import gc , psutil
import time , argparse
from scripts.data_util.ModelData import (
     DataBlock ,block_process ,block_hist_norm,
     path_block_data,path_norm_data,block_mask)
from scripts.data_util.DataTank import DataTank
from scripts.function.basic import *
from scripts.util.environ import get_logger , DIR_data
from scripts.util.basic import timer
 
logger = get_logger()
DIR_block  = f'{DIR_data}/block_data'
DIR_hist_norm = f'{DIR_data}/hist_norm'

_save_start_dt , _save_end_dt = 20070101 , None
general_param = {
    'start_dt' : None , 'end_dt' : None , 'mask' : True ,
} # 'start_dt' : 20150101 , 'end_dt' : 20150331 , 'mask' : True ,
process_param = {
    'trade_day' : {
        'DB_key'  : 'trade_day' , 
        'DB_path' : 'day/trade' ,
        'feature' : ['adjfactor', 'close', 'high', 'low', 'open', 'volume', 'vwap'] , 'process_method' : 'adj_order' ,
    },
}

In [9]:
x_trade_norm_dict = dict()
key = 'trade_day'
param = process_param['trade_day']
                

In [10]:
tt1 = time.time()
print(f'{time.ctime()} : {key} start ...')

blocks = []
db_key = param['DB_key']
db_path = [param['DB_path']] if isinstance(param['DB_path'] , str) else param['DB_path']
kwargs = {**param , **general_param}

for path in db_path:
    with timer(f'{key} blocks reading {path} Data1D\'s') as t:
        blocks.append(DataBlock().from_db(db_key , path , **kwargs))

with timer(f'{key} blocks merging') as t:
    ThisBlock = DataBlock().merge_others(blocks)
    del blocks
    gc.collect()

with timer(f'{key} blocks process') as t:
    ThisBlock = block_mask(ThisBlock , **kwargs)
    ThisBlock = block_process(ThisBlock , **kwargs)

with timer(f'{key} blocks saving ') as t:
    ThisBlock.save(path_block_data(key) , start_dt=_save_start_dt , end_dt=_save_end_dt)

with timer(f'{key} blocks norming') as t:
    #ThisBlock = DataBlock().read_npz(path_block_data(key))
    block_hist_norm(ThisBlock , key , path_norm_data(key) , **param)

tt2 = time.time()
print(f'{time.ctime()} : {key} finished! Cost {tt2-tt1:.2f} Seconds')

del ThisBlock
gc.collect()

t2 = time.time()

Mon Jan  8 22:05:42 2024 : trade_day start ...
trade_day blocks reading day/trade Data1D's ...... cost 26.71 secs
trade_day blocks merging ...... cost 0.14 secs
trade_day blocks process ...... cost 5.87 secs
trade_day blocks saving  ...... cost 24.41 secs
trade_day blocks norming ...... cost 23.81 secs
Mon Jan  8 22:07:02 2024 : trade_day finished! Cost 80.94 Seconds


In [7]:
kwargs

{'DB_key': 'DB_trade_day.h5',
 'DB_path': 'day/trade',
 'feature': ['adjfactor', 'close', 'high', 'low', 'open', 'volume', 'vwap'],
 'process_method': 'adj_order',
 'start_dt': None,
 'end_dt': None,
 'mask': True}

In [6]:
blocks[0]

uninitiate <class 'scripts.data_util.ModelData.DataBlock'>